In [1]:
# pip install tensorflow

In [1]:
import time
import random
import pandas as pd
import numpy as np
import os

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, MaxPooling2D, Conv2D, Dropout, Lambda, Dense, Flatten, Activation, Input, Embedding, BatchNormalization
from tensorflow.keras.initializers import glorot_normal, Zeros, TruncatedNormal
from tensorflow.keras.regularizers import l2


from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy


from tensorflow.keras.optimizers import Adam
from collections import defaultdict
import math

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
class FeaturesEmbedding(Layer):
    '''
    임베딩 레이어입니다.
    - 만약 피처(feature) 3개가 각각 10개, 20개, 30개의 고유값을 가진다면 feature_dims는 [10, 20, 30] 형태를 띄게 됩니다.
    - 전체 임베딩을 해야 할 개수는 10+20+30 = 60이므로 '60 x 임베딩_차원_크기'의 행렬이 생성되게 됩니다.
    '''
    def __init__(self, field_dims, embed_dim, **kwargs):
        super(FeaturesEmbedding, self).__init__(**kwargs)
        self.total_dim = sum(field_dims)
        self.embed_dim = embed_dim
        ## int 64로 바꿔주자
        self.offsets = np.array((0, *np.cumsum(field_dims)[:-1]), dtype=np.int64)

        # build에서 초기화하지 않고 __init__에서 정의합니다.
        # embeddings_initializer를 사용하여 초기화 방법을 지정합니다.
        self.embedding = tf.keras.layers.Embedding(
            input_dim=self.total_dim,
            output_dim=self.embed_dim,
            embeddings_initializer='glorot_uniform'
        )

    def call(self, x):
        # 들어온 입력의 임베딩을 가져니다.
        x = x + tf.constant(self.offsets)
        return self.embedding(x)

In [4]:
class MultiLayerPerceptron(Layer):
    '''
    DNN 레이어입니다.
    - Tensorflow Keras에서는 Dense 레이어를 쌓아올린 구조입니다.
    - 필요에 따라 배치 정규화도 사용할 수 있습니다.
    '''
    def __init__(self, input_dim, hidden_units, activation='relu', l2_reg=0, dropout_rate=0, use_bn=False, init_std=0.0001, output_layer=True):
        super(MultiLayerPerceptron, self).__init__()
        self.dropout_rate = dropout_rate
        self.use_bn = use_bn
        hidden_units = [input_dim] + list(hidden_units)
        if output_layer:
            hidden_units += [1]
        # Dense layer를 쌓아올립니다.
        self.linears = [Dense(units, activation=None, kernel_initializer=tf.random_normal_initializer(stddev=init_std),
                              kernel_regularizer=tf.keras.regularizers.l2(l2_reg)) for units in hidden_units[1:]]
        # 활성화 함수를 세팅합니다.
        self.activation = tf.keras.layers.Activation(activation)
        # 필요하다면 배치정규화도 진행합니다.
        if self.use_bn:
            self.bn = [BatchNormalization() for _ in hidden_units[1:]]
        self.dropout = Dropout(dropout_rate)

    def call(self, inputs, training=False):
        x = inputs
        for i in range(len(self.linears)):
            # input data가 들어오면 layer를 돌면서 벡터 값을 가져오게 됩니다.
            x = self.linears[i](x)
            if self.use_bn:
                x = self.bn[i](x, training=training)
            # 각 layer마다 나온 벡터 값에 활성화 함수와 dropout을 적용시켜 비선형성 구조와 과적합을 방지합니다.
            x = self.activation(x)
            x = self.dropout(x, training=training)
        return x

In [5]:
class MultiHeadSelfAttention(Layer):
    '''
    멀티 헤드 셀프 어텐션 레이어입니다.
    - 위에 작성한 수식과 같이 동작됩니다.
    - 필요에 따라 잔차 연결(residual connection)도 진행합니다.
    '''
    def __init__(self, att_embedding_size=8, head_num=2, use_res=True, scaling=False, seed=1024, **kwargs):
        if head_num <= 0:
            raise ValueError('head_num must be a int > 0')
        self.att_embedding_size = att_embedding_size
        self.head_num = head_num
        self.use_res = use_res
        self.seed = seed
        self.scaling = scaling
        super(MultiHeadSelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        if len(input_shape) != 3:
            raise ValueError(
                "Unexpected inputs dimensions %d, expect to be 3 dimensions" % (len(input_shape)))
        embedding_size = int(input_shape[-1])
        # 쿼리에 해당하는 매트릭스입니다.
        self.W_Query = self.add_weight(name='query', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed))
        # 키에 해당되는 매트릭스입니다.
        self.W_key = self.add_weight(name='key', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                     dtype=tf.float32,
                                     initializer=TruncatedNormal(seed=self.seed + 1))
        # 값(value)에 해당되는 매트릭스입니다.
        self.W_Value = self.add_weight(name='value', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                       dtype=tf.float32,
                                       initializer=TruncatedNormal(seed=self.seed + 2))
        # 필요하다면 잔차 연결도 할 수 있습니다.
        if self.use_res:
            self.W_Res = self.add_weight(name='res', shape=[embedding_size, self.att_embedding_size * self.head_num],
                                         dtype=tf.float32,
                                         initializer=TruncatedNormal(seed=self.seed))

        super(MultiHeadSelfAttention, self).build(input_shape)

    def call(self, inputs, **kwargs):
        if K.ndim(inputs) != 3:
            raise ValueError("Unexpected inputs dimensions %d, expect to be 3 dimensions" % (K.ndim(inputs)))

        # 입력이 들어오면 쿼리, 키, 값(value)에 매칭되어 각각의 값을 가지고 옵니다.
        querys = tf.tensordot(inputs, self.W_Query, axes=(-1, 0))
        keys = tf.tensordot(inputs, self.W_key, axes=(-1, 0))
        values = tf.tensordot(inputs, self.W_Value, axes=(-1, 0))

        # 헤드 개수에 따라 데이터를 분리해줍니다.
        querys = tf.stack(tf.split(querys, self.head_num, axis=2))
        keys = tf.stack(tf.split(keys, self.head_num, axis=2))
        values = tf.stack(tf.split(values, self.head_num, axis=2))

        # 쿼리와 키를 먼저 곱해줍니다. 위 이미지의 식 (5)와 같습니다.
        inner_product = tf.matmul(querys, keys, transpose_b=True)
        if self.scaling:
            inner_product /= self.att_embedding_size ** 0.5
        self.normalized_att_scores =  tf.nn.softmax(inner_product)

        # 쿼리와 키에서 나온 어텐션 값을 값(value)에 곱해줍니다. 식 (6)과 같습니다.
        result = tf.matmul(self.normalized_att_scores, values)
        # 식 (7)과 같이 쪼개어진 멀테 헤드를 모아줍니다.
        result = tf.concat(tf.split(result, self.head_num, ), axis=-1)
        result = tf.squeeze(result, axis=0)

        if self.use_res:
            result += tf.tensordot(inputs, self.W_Res, axes=(-1, 0))
        result = tf.nn.relu(result)

        # 그 결과 값을 리턴합니다.

        return result

    def compute_output_shape(self, input_shape):

        return (None, input_shape[1], self.att_embedding_size * self.head_num)

    def get_config(self, ):
        config = {'att_embedding_size': self.att_embedding_size, 'head_num': self.head_num, 'use_res': self.use_res,'seed': self.seed}
        base_config = super(MultiHeadSelfAttention, self).get_config()
        base_config.update(config)
        return base_config

# 바꿀 부분

이 밑에있는 모델 정의를 새로운 모델로 해주셔야합니다

In [6]:
from tensorflow.keras.layers import LayerNormalization

class AutoIntMLP(Layer):
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2, att_res=True, dnn_hidden_units=(32, 32), dnn_activation='gelu',
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=True, dnn_dropout=0.4, init_std=0.0001):
        super(AutoIntMLP, self).__init__()
        self.embedding = FeaturesEmbedding(field_dims, embedding_size)
        self.num_fields = len(field_dims)
        self.embedding_size = embedding_size

        self.final_layer = Dense(1, use_bias=False, kernel_initializer=tf.random_normal_initializer(stddev=init_std))

        self.dnn = tf.keras.Sequential()
        for units in dnn_hidden_units:
            self.dnn.add(Dense(units, activation=dnn_activation,
                               kernel_regularizer=tf.keras.regularizers.l2(l2_reg_dnn),
                               kernel_initializer=tf.random_normal_initializer(stddev=init_std)))
            if dnn_use_bn:
                self.dnn.add(BatchNormalization())
            self.dnn.add(Activation(dnn_activation))
            if dnn_dropout > 0:
                self.dnn.add(Dropout(dnn_dropout))
        self.dnn.add(Dense(1, kernel_initializer=tf.random_normal_initializer(stddev=init_std)))

        self.int_layers = [MultiHeadSelfAttention(att_embedding_size=embedding_size, head_num=att_head_num, use_res=att_res) for _ in range(att_layer_num)]

        # [개선] Layer Normalization 추가 (Attention 층 개수만큼)
        # Attention 네트워크의 학습 안정성을 위해 필수적입니다.
        self.layer_norms = [LayerNormalization() for _ in range(att_layer_num)]

    def call(self, inputs, training=False):
        embed_x = self.embedding(inputs)
        dnn_embed = tf.reshape(embed_x, shape=(-1, self.embedding_size * self.num_fields))

        att_input = embed_x

        for layer, ln in zip(self.int_layers, self.layer_norms):
            att_output = layer(att_input)
            # [개선] Add & Norm (Transformer 구조 적용)
            # MultiHeadSelfAttention 내부에서 이미 Residual(Add)이 있다면 Norm만 적용
            # 만약 내부 res=False라면: att_input = ln(att_input + att_output) 형태가 됨
            att_input = ln(att_output)

        att_output = Flatten()(att_input)
        att_output = self.final_layer(att_output)

        # [개선] training 인자 전달 (Dropout, BN 적용을 위해 필수)
        dnn_output = self.dnn(dnn_embed, training=training)
        y_pred = tf.keras.activations.sigmoid(att_output + dnn_output)

        return y_pred

In [7]:
def get_DCG(ranklist, y_true):
    dcg = 0.0
    for i in range(len(ranklist)):
        item = ranklist[i]
        if item in y_true:
            dcg += 1.0 / math.log(i + 2)
    return  dcg

def get_IDCG(ranklist, y_true):
    idcg = 0.0
    i = 0
    for item in y_true:
        if item in ranklist:
            idcg += 1.0 / math.log(i + 2)
            i += 1
    return idcg

def get_NDCG(ranklist, y_true):
    '''NDCG 평가 지표'''
    ranklist = np.array(ranklist).astype(int)
    y_true = np.array(y_true).astype(int)
    dcg = get_DCG(ranklist, y_true)
    idcg = get_IDCG(y_true, y_true)
    if idcg == 0:
        return 0
    return round( (dcg / idcg), 5)

def get_hit_rate(ranklist, y_true):
    '''hitrate 평가 지표'''
    c = 0
    for y in y_true:
        if y in ranklist:
            c += 1
    return round( c / len(y_true), 5 )

In [8]:
def test_model(model, test_df):
    '''모델 테스트'''
    user_pred_info = defaultdict(list)
    total_rows = len(test_df)
    for i in range(0, total_rows, batch_size):
        features = test_df.iloc[i:i + batch_size, :-1].values
        y_pred = model.predict(features, verbose=False)
        ##
        y_pred = y_pred.flatten()
        for feature, p in zip(features, y_pred):
            u_i = feature[:2]
            user_pred_info[int(u_i[0])].append((int(u_i[1]), float(p)))
    return user_pred_info

# 학습용 데이터 만들기

In [9]:
project_path = os.path.abspath(os.getcwd())
data_dir_nm = 'data'
movielens_dir_nm = 'ml-1m'
model_dir_nm = 'model'
data_path = f"{project_path}/{data_dir_nm}"

In [10]:
# 1. 데이터 불러오기
# csv 데이터이므로 read_csv로 가져옵니다.
movielens_rcmm = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/movielens_rcmm_v4.csv", dtype=str)
print(movielens_rcmm.shape)
movielens_rcmm.head()

(5287350, 19)


,user_id,movie_id,movie_decade,movie_year,rating_year,rating_month,genre1,genre2,genre3,gender,age,occupation,zip,movie_pop,user_act,release_lag,age_gender,age_genre,label
0,1,1193,1970s,1975,2001,1,Drama,no,no,F,1,10,48067,9,0,5,1_F,1_Drama,1
1,1,3408,2000s,2000,2001,1,Drama,no,no,F,1,10,48067,9,0,0,1_F,1_Drama,1
2,1,2355,1990s,1998,2001,1,Animation,Children's,Comedy,F,1,10,48067,9,0,0,1_F,1_Animation,1
3,1,1287,1950s,1959,2001,1,Action,Adventure,Drama,F,1,10,48067,8,0,8,1_F,1_Action,1
4,1,2804,1980s,1983,2001,1,Comedy,Drama,no,F,1,10,48067,9,0,3,1_F,1_Comedy,1


In [11]:
label_encoders = {col: LabelEncoder() for col in movielens_rcmm.columns[:-1]} # label은 제외

for col, le in label_encoders.items():
    movielens_rcmm[col] = le.fit_transform(movielens_rcmm[col])

In [12]:
movielens_rcmm.head()

,user_id,movie_id,movie_decade,movie_year,rating_year,rating_month,genre1,genre2,genre3,gender,age,occupation,zip,movie_pop,user_act,release_lag,age_gender,age_genre,label
0,0,212,6,55,1,0,7,17,15,0,0,2,1588,9,0,12,2,25,1
1,0,2619,9,80,1,0,7,17,15,0,0,2,1588,9,0,0,2,25,1
2,0,1450,8,78,1,0,2,2,2,0,0,2,1588,9,0,0,2,20,1
3,0,313,4,39,1,0,0,0,4,0,0,2,1588,8,0,15,2,18,1
4,0,1949,7,63,1,0,4,6,15,0,0,2,1588,9,0,10,2,22,1


In [13]:
movielens_rcmm['label'] = movielens_rcmm['label'].astype(np.float32)

In [14]:
# 3. 학습 데이터와 테스트데이터로 분리, 0.2 정도로 분리
train_df, test_df = train_test_split(movielens_rcmm, test_size=0.2, random_state=42)

In [15]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4229880 entries, 3115137 to 1692743
Data columns (total 19 columns):
 #   Column        Dtype  
---  ------        -----  
 0   user_id       int64  
 1   movie_id      int64  
 2   movie_decade  int64  
 3   movie_year    int64  
 4   rating_year   int64  
 5   rating_month  int64  
 6   genre1        int64  
 7   genre2        int64  
 8   genre3        int64  
 9   gender        int64  
 10  age           int64  
 11  occupation    int64  
 12  zip           int64  
 13  movie_pop     int64  
 14  user_act      int64  
 15  release_lag   int64  
 16  age_gender    int64  
 17  age_genre     int64  
 18  label         float32
dtypes: float32(1), int64(18)
memory usage: 629.3 MB


In [16]:
# 필요 컬럼들과 레이블 정의
# 필드의 각 고유 개수를 정의하는 field_dims를 정의합니다. 이는  임베딩 때 활용됩니다.
u_i_feature = ['user_id', 'movie_id']
meta_features = ['movie_decade', 'movie_year', 'rating_year', 'rating_month', 'genre1','genre2', 'genre3', 'gender', 'age', 'occupation', 'zip', 'movie_pop', 'user_act', 'release_lag','age_gender', 'age_genre']
label = 'label'
field_dims = np.max(movielens_rcmm[u_i_feature + meta_features].astype(np.int64).values, axis=0) + 1
field_dims

array([6040, 3883,   10,   81,    4,   12,   18,   18,   16,    2,    7,
         21, 3439,   10,   10,   17,   14,  126])

In [17]:
# 에포크, 학습률, 드롭아웃, 배치사이즈, 임베딩 크기 등 정의
epochs=5
learning_rate= 0.001
dropout= 0.5 # 기존 0.4
batch_size = 4096
embed_dim= 64 # 기존 16

# 고칠부분

이부분도 AutoIntMLP를 가져와주고 DNN 레이어가 붙은 파라미터를 추가해줍니다

In [18]:
class AutoIntMLPModel(Model):
    def __init__(self, field_dims, embedding_size, att_layer_num=3, att_head_num=2,
                 att_res=True, dnn_hidden_units=(32, 32), dnn_activation='gelu',
                 l2_reg_dnn=0, l2_reg_embedding=1e-5, dnn_use_bn=True,
                 dnn_dropout=0.4, init_std=0.0001):
        super(AutoIntMLPModel, self).__init__()
        self.autoInt_layer = AutoIntMLP(
            field_dims=field_dims,
            embedding_size=embedding_size,
            att_layer_num=att_layer_num,
            att_head_num=att_head_num,
            att_res=att_res,
            dnn_hidden_units=dnn_hidden_units,
            dnn_activation=dnn_activation,
            l2_reg_dnn=l2_reg_dnn,
            l2_reg_embedding=l2_reg_embedding,
            dnn_use_bn=dnn_use_bn,
            dnn_dropout=dnn_dropout,
            init_std=init_std
        )

    def call(self, inputs, training=False):
        return self.autoInt_layer(inputs, training=training)

In [19]:
# 모델 정의
autoIntMLP_model = AutoIntMLPModel(
    field_dims=field_dims,
    embedding_size=embed_dim,
    att_layer_num=3,
    att_head_num=2,
    att_res=True,
    dnn_hidden_units=(32, 32),                    # 추가: DNN 은닉층 구조
    dnn_activation='gelu', #relu 대신 gelu로 변경   # 추가: 활성화 함수
    l2_reg_dnn=0,
    l2_reg_embedding=1e-5,
    dnn_use_bn=True, # True로 변경,과적합 방지.
    dnn_dropout=dropout,
    init_std=0.0001
)

In [20]:
# 옵티마이저, 오차함수 정의
optimizer = Adam(learning_rate=learning_rate)
loss_fn = BinaryCrossentropy(from_logits=False)

In [21]:
autoIntMLP_model.compile(optimizer=optimizer, loss=loss_fn, metrics=['binary_crossentropy'])

In [22]:
history = autoIntMLP_model.fit(train_df[u_i_feature + meta_features], train_df[label], epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/5
930/930 ━━━━━━━━━━━━━━━━━━━━ 76s 65ms/step - binary_crossentropy: 0.0176 - loss: 0.0176 - val_binary_crossentropy: 2.1047e-06 - val_loss: 2.1047e-06
Epoch 2/5
930/930 ━━━━━━━━━━━━━━━━━━━━ 53s 57ms/step - binary_crossentropy: 1.6806e-06 - loss: 1.6806e-06 - val_binary_crossentropy: 7.6560e-07 - val_loss: 7.6560e-07
Epoch 3/5
930/930 ━━━━━━━━━━━━━━━━━━━━ 53s 57ms/step - binary_crossentropy: 6.7978e-07 - loss: 6.7978e-07 - val_binary_crossentropy: 3.7949e-07 - val_loss: 3.7949e-07
Epoch 4/5
930/930 ━━━━━━━━━━━━━━━━━━━━ 53s 57ms/step - binary_crossentropy: 3.4998e-07 - loss: 3.4998e-07 - val_binary_crossentropy: 2.0863e-07 - val_loss: 2.0863e-07
Epoch 5/5
930/930 ━━━━━━━━━━━━━━━━━━━━ 53s 57ms/step - binary_crossentropy: 1.9685e-07 - loss: 1.9685e-07 - val_binary_crossentropy: 1.2097e-07 - val_loss: 1.2097e-07


In [23]:
# 사용자에게 예측된 정보를 저장하는 딕셔너리
user_pred_info = {}
# top10개
top = 10
# 테스트 값을 가지고 옵니다.
mymodel_user_pred_info = test_model(autoIntMLP_model, test_df)
# 사용자마다 돌면서 예측 데이터 중 가장 높은 top 10만 가져옵니다.
for user, data_info in tqdm(mymodel_user_pred_info.items(), total=len(mymodel_user_pred_info), position=0, leave=True):
    ranklist = sorted(data_info, key=lambda s : s[1], reverse=True)[:top]
    ranklist = list(dict.fromkeys([r[0] for r in ranklist]))
    user_pred_info[str(user)] = ranklist
# 원본 테스트 데이터에서 label이 1인 사용자 별 영화 정보를 가져옵니다.
test_data = test_df[test_df['label']==1].groupby('user_id')['movie_id'].apply(list)

100%|██████████| 6040/6040 [00:00<00:00, 14741.43it/s]


In [24]:
mymodel_ndcg_result = {}
mymodel_hitrate_result = {}

# 모델 예측값과 원본 테스트 데이터를 비교해서 어느정도 성능이 나왔는지 NDCG와 Hitrate를 비교합니다.

# NDCG
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # NDCG 값 구하기
    user_ndcg = get_NDCG(mymodel_pred, testset)

    mymodel_ndcg_result[user] = user_ndcg

# Hitrate
for user, data_info in tqdm(test_data.items(), total=len(test_data), position=0, leave=True):
    mymodel_pred = user_pred_info.get(str(user))

    testset = list(set(np.array(data_info).astype(int)))
    mymodel_pred = mymodel_pred[:top]

    # hitrate 값 구하기
    user_hitrate = get_hit_rate(mymodel_pred, testset)

    # 사용자 hitrate 결과 저장
    mymodel_hitrate_result[user] = user_hitrate

100%|██████████| 5981/5981 [00:00<00:00, 51114.61it/s]


In [25]:
print(" mymodel ndcg : ", round(np.mean(list(mymodel_ndcg_result.values())), 5))
print(" mymodel hitrate : ", round(np.mean(list(mymodel_hitrate_result.values())), 5))

 mymodel ndcg :  0.77703
 mymodel hitrate :  0.70698


In [29]:
np.save('/content/drive/MyDrive/Colab Notebooks/Data/field_dims2.npy', field_dims)

In [30]:
autoIntMLP_model.save_weights('/content/drive/MyDrive/Colab Notebooks/autoIntMLP_model5_weights.weights.h5')

In [31]:
import joblib

# 모델 객체를 pickled binary file 형태로 저장
joblib.dump(label_encoders, '/content/drive/MyDrive/Colab Notebooks/Data/label_encoders2.pkl')

['/content/drive/MyDrive/Colab Notebooks/Data/label_encoders2.pkl']